In [ ]:
!git clone https://github.com/facebookresearch/audiocraft.git
%cd audiocraft
!pip install -e .
!pip install dora-search
!pip install jsonlines

In [2]:
# download dataset.tar.gz and extract

import urllib.request
import tarfile
import glob
import os
import json, jsonlines

# URL and destination folder
url = "path/to/dataset"
destination_folder = "dataset"
os.makedirs(destination_folder, exist_ok=True)

# Download tar.gz file
tar_gz_path = os.path.join(destination_folder, "dataset_name.tar.gz")
urllib.request.urlretrieve(url, tar_gz_path)

# Extract tar.gz into destination folder
with tarfile.open(tar_gz_path, 'r:gz') as tar:
    tar.extractall(path=destination_folder)

# kill text files
txt_files = glob.glob(os.path.join(destination_folder, '*.txt'))
for txt_file in txt_files:
    os.remove(txt_file)

# download yaml and jsonl

urls_destinations = {
    "path/to/test.jsonl": "/workspace/audiocraft/egs/eval/data.jsonl",
    "path/to/train.jsonl": "/workspace/audiocraft/egs/train/data.jsonl"
}
os.makedirs("/workspace/audiocraft/egs/train", exist_ok=True)
os.makedirs("/workspace/audiocraft/egs/eval", exist_ok=True)

for url, destination in urls_destinations.items():
    urllib.request.urlretrieve(url, destination)

urllib.request.urlretrieve("path/to/train.yaml", "/workspace/audiocraft/config/dset/audio/train.yaml")

In [18]:
# fix paths in jsonl if needed
for jsonl_file in ["/workspace/audiocraft/egs/train/data.jsonl","/workspace/audiocraft/egs/eval/data.jsonl"]:
    temp_list = []
    with open(jsonl_file, 'r') as infile:
        for line in infile:
            obj = json.loads(line)
            file_name = os.path.basename(obj["path"])
            new_path = os.path.join(destination_folder, file_name)
            obj["path"] = new_path
            temp_list.append(json.dumps(obj))

    with open(jsonl_file, 'w') as outfile:
        outfile.write('\n'.join(temp_list) + '\n')

In [ ]:
# train
%env USER=seb
command = (
    "dora -P audiocraft run -d "
    " solver=musicgen/musicgen_base_32khz"
    " model/lm/model_scale=large"
    " continue_from=//pretrained/facebook/musicgen-large"
    " conditioner=text2music"
    " dset=audio/train"
    " dataset.num_workers=2"
    " dataset.valid.num_samples=1"
    " dataset.batch_size=4"
    " schedule.cosine.warmup=8"
    " optim.optimizer=adamw"
    " optim.lr=1e-4"
    " optim.epochs=5"
    " optim.adam.weight_decay=0.01"
    " fsdp.use=true"
    " autocast=false"
)
!{command}


In [ ]:
#save the model
folder_to_save_checkpoints_in = "/workspace/checkpoint"
os.makedirs(folder_to_save_checkpoints_in, exist_ok=True)

import os
root_dir = "/tmp/audiocraft_seb/xps/"
subfolders = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
joined_paths = [os.path.join(root_dir, subfolder) for subfolder in subfolders]
SIG = max(joined_paths, key=os.path.getmtime)

from audiocraft.utils import export
from audiocraft import train
xp = train.main.get_xp_from_sig(SIG)
export.export_lm(xp.folder / 'checkpoint.th', os.path.join(folder_to_save_checkpoints_in, 'state_dict.bin'))
export.export_pretrained_compression_model('facebook/encodec_32khz', os.path.join(folder_to_save_checkpoints_in, 'compression_state_dict.bin'))